### 관광지명을 통해 주소를 가져오기 위한 동적 크롤링

In [ ]:
# final_2022_제주 파일로 시연 가능
# 수도권, 충청, 경상, 제주, 강원, 전라 
# 전체 데이터 파일을 6지역으로 나눠서 동적 크롤링을 통해
# 카카오맵에서 주소 가져와서 파일에 저장하기

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# 크롬 드라이버 경로 설정
# driver_path = "크롬 드라이버 경로"

df = pd.read_excel('./final_2022_제주.xlsx')

tourist_name = list(df['관광지명'])

add_list = []

for i in range(len(tourist_name)):

    # 크롬 드라이버 실행
    driver = webdriver.Chrome()

    # 카카오맵 접속
    driver.get("https://map.kakao.com/")

    # 검색어 입력
    search_input = driver.find_element("id", "search.keyword.query")
    search_input.send_keys(tourist_name[i])

    time.sleep(2)

    search_pass = driver.find_element(By.XPATH, '//*[@id="dimmedLayer"]')
    search_pass.click()

    time.sleep(2)

    # 검색 버튼 클릭
    search_button = driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]')
    search_button.click()

    time.sleep(2)

    # # 주소 가져오기
    try:
        address = driver.find_element(By.XPATH, '//*[@id="info.search.place.list"]/li[1]/div[5]/div[2]/p[1]').text
        add_list.append(address)

    except:
        add_list.append('NONE')
    # search_input.clear()

    # # 드라이버 종료
    driver.quit()


df['주소'] = ''

for i in range(len(add_list)):
    df['주소'][i] = add_list[i]


df.to_excel('./final_2022_제주.xlsx', index=False)


### 카카오맵 api 통해 경도, 위도 가져오기

In [ ]:
# 위에서 가져온 주소를 카카오맵 api를 통해 경도, 위도 찾는 함수 구현
# https://apis.map.kakao.com/ => 접속 후 REST API 애플리케이션 추가하기 

import json
import requests

api_key = '679acde1aaa81383cfa1d85d0b7ca9d1'


def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return [float(match_first['x']), float(match_first['y'])]

In [ ]:
# 위에 동적 크롤링을 통해 제주 파일에 주소를 가져와야 실행 가능!

jeju_df = pd.read_excel('./final_2022_제주.xlsx')

jeju_add = jeju_df['주소']

for i in range(len(jeju_add)):
    try :
        jeju_df['경도'][i] = addr_to_lat_lon(jeju_add[i])[0]
        jeju_df['위도'][i] = addr_to_lat_lon(jeju_add[i])[1]
    except:
        jeju_df['경도'][i] = 0
        jeju_df['위도'][i] = 0

jeju_df.to_excel('./final_2022_제주.xlsx')

### map 라이브러리를 통해 지역별 지도 그리기

In [17]:
# 6 지역별 지도 그리기 - 수도권
import pandas as pd

capital_df = pd.read_excel('./final_2022_서울경기인천_완성본.xlsx')

capital = capital_df[['관광지명', '위도', '경도', '주소', '2022년']]
capital_df

import folium

center_coords = [37.56, 126.90]
zoom_level = 9

capital_map = folium.Map(location=center_coords, zoom_start=zoom_level, tiles='cartodbpositron')


for idx in capital.index:
    lat = capital.loc[idx, '위도']
    lng = capital.loc[idx, '경도']
    name = capital.loc[idx, '관광지명']
    peo_num = capital.loc[idx, '2022년']//10000

    # 관광지 방문객수 기준 크기 선택을 위한 조건문
    if peo_num >= 80:
        size = 40
    elif peo_num >= 60:
        size = 30
    elif peo_num >= 40:
        size = 20
    elif peo_num >= 20:
        size = 10
    else:
        size = 5

    folium.CircleMarker(
        location=[lat, lng],
        color= '#86B6F6',
        fill = True,
        fill_color = '#86B6F6', 
        fill_opacity = 0.5,
        weight = 1,
        radius = size,
        popup = name
    ).add_to(capital_map)

capital_map

In [8]:
# 6 지역별 지도 그리기 - 강원도
import pandas as pd

gangwon_df = pd.read_excel('./final_2022_강원_완성본.xlsx')

gangwon = gangwon_df[['관광지명', '위도', '경도', '주소', '2022년']]
gangwon_df

import folium

center_coords = [37.53, 128.59]
zoom_level = 10.5

gangwon_map = folium.Map(location=center_coords, zoom_start=zoom_level, tiles='cartodbpositron')


for idx in gangwon.index:
    lat = gangwon.loc[idx, '위도']
    lng = gangwon.loc[idx, '경도']
    name = gangwon.loc[idx, '관광지명']
    peo_num = gangwon.loc[idx, '2022년']//10000

    # 관광지 방문객수 기준 크기 선택을 위한 조건문
    if peo_num >= 80:
        size = 40
    elif peo_num >= 60:
        size = 30
    elif peo_num >= 40:
        size = 20
    elif peo_num >= 20:
        size = 10
    else:
        size = 5

    folium.CircleMarker(
        location=[lat, lng],
        color= '#0D9276',
        fill = True,
        fill_color = '#0D9276', 
        fill_opacity = 0.5,
        weight = 1,
        radius = size,
        popup = name
    ).add_to(gangwon_map)

gangwon_map

In [12]:
# 6 지역별 지도 그리기 - 충청도
import pandas as pd

chungcheong_df = pd.read_excel('./final_2022_충청_완성본.xlsx')

chungcheong = chungcheong_df[['관광지명', '위도', '경도', '주소', '2022년']]
chungcheong_df

import folium

center_coords = [36.71, 126.8]
zoom_level = 9

chungcheong_map = folium.Map(location=center_coords, zoom_start=zoom_level, tiles='cartodbpositron')


for idx in chungcheong.index:
    lat = chungcheong.loc[idx, '위도']
    lng = chungcheong.loc[idx, '경도']
    name = chungcheong.loc[idx, '관광지명']
    peo_num = chungcheong.loc[idx, '2022년']//10000

    # 관광지 방문객수 기준 크기 선택을 위한 조건문
    if peo_num >= 80:
        size = 40
    elif peo_num >= 60:
        size = 30
    elif peo_num >= 40:
        size = 20
    elif peo_num >= 20:
        size = 10
    else:
        size = 5

    folium.CircleMarker(
        location=[lat, lng],
        color= '#E7B10A',
        fill = True,
        fill_color = '#E7B10A', 
        fill_opacity = 0.5,
        weight = 1,
        radius = size,
        popup = name
    ).add_to(chungcheong_map)

chungcheong_map

In [14]:
# 6 지역별 지도 그리기 - 경상도
import pandas as pd

gyeongsang_df = pd.read_excel('./final_2022_경상_완성본.xlsx')

gyeongsang = gyeongsang_df[['관광지명', '위도', '경도', '주소', '2022년']]
gyeongsang_df

import folium

center_coords = [35.46, 128.21]
zoom_level = 9

gyeongsang_map = folium.Map(location=center_coords, zoom_start=zoom_level, tiles='cartodbpositron')


for idx in gyeongsang.index:
    lat = gyeongsang.loc[idx, '위도']
    lng = gyeongsang.loc[idx, '경도']
    name = gyeongsang.loc[idx, '관광지명']
    peo_num = gyeongsang.loc[idx, '2022년']//10000

    # 관광지 방문객수 기준 크기 선택을 위한 조건문
    if peo_num >= 80:
        size = 40
    elif peo_num >= 60:
        size = 30
    elif peo_num >= 40:
        size = 20
    elif peo_num >= 20:
        size = 10
    else:
        size = 5

    folium.CircleMarker(
        location=[lat, lng],
        color= '#A459D1',
        fill = True,
        fill_color = '#A459D1', 
        fill_opacity = 0.5,
        weight = 1,
        radius = size,
        popup = name
    ).add_to(gyeongsang_map)

gyeongsang_map

In [15]:
# 6 지역별 지도 그리기 - 전라도
import pandas as pd

jeolla_df = pd.read_excel('./final_2022_전라_완성본.xlsx')

jeolla = jeolla_df[['관광지명', '위도', '경도', '주소', '2022년']]
jeolla_df

import folium

center_coords = [34.86, 126.99]
zoom_level = 9

jeolla_map = folium.Map(location=center_coords, zoom_start=zoom_level, tiles='cartodbpositron')


for idx in jeolla.index:
    lat = jeolla.loc[idx, '위도']
    lng = jeolla.loc[idx, '경도']
    name = jeolla.loc[idx, '관광지명']
    peo_num = jeolla.loc[idx, '2022년']//10000

    # 관광지 방문객수 기준 크기 선택을 위한 조건문
    if peo_num >= 80:
        size = 40
    elif peo_num >= 60:
        size = 30
    elif peo_num >= 40:
        size = 20
    elif peo_num >= 20:
        size = 10
    else:
        size = 5

    folium.CircleMarker(
        location=[lat, lng],
        color= '#F266AB',
        fill = True,
        fill_color = '#F266AB', 
        fill_opacity = 0.5,
        weight = 1,
        radius = size,
        popup = name
    ).add_to(jeolla_map)

jeolla_map

In [16]:
# 6 지역별 지도 그리기 - 제주도
import pandas as pd

jeju_df = pd.read_excel('./final_2022_제주_완성본.xlsx')

jeju = jeju_df[['관광지명', '위도', '경도', '주소', '2022년']]
jeju_df

import folium

center_coords = [33.355, 126.543]
zoom_level = 10.5

jeju_map = folium.Map(location=center_coords, zoom_start=zoom_level, tiles='cartodbpositron')


for idx in jeju.index:
    lat = jeju.loc[idx, '위도']
    lng = jeju.loc[idx, '경도']
    name = jeju.loc[idx, '관광지명']
    peo_num = jeju.loc[idx, '2022년']//10000

    # 관광지 방문객수 기준 크기 선택을 위한 조건문
    if peo_num >= 80:
        size = 40
    elif peo_num >= 60:
        size = 30
    elif peo_num >= 40:
        size = 20
    elif peo_num >= 20:
        size = 10
    else:
        size = 5

    folium.CircleMarker(
        location=[lat, lng],
        color= 'orange',
        fill = True,
        fill_color = 'orange', 
        fill_opacity = 0.5,
        weight = 1,
        radius = size,
        popup = name
    ).add_to(jeju_map)

jeju_map